In [2]:
# 최적화 코드
# light gbm 사용하기
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.multioutput import MultiOutputRegressor

dataset = pd.read_csv('train.csv')
X = dataset.iloc[:,1:57]
y = dataset.iloc[:,57:]

X.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_47', 'X_48', 'X_37'], axis = 1, inplace = True) 

sc = MinMaxScaler()
X = sc.fit_transform(X)

# 훈련데이터, 테스트 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)



# 평가방식
from sklearn import metrics
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [3]:
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(x_train, y_train)
lasso_preds = lasso.predict(x_test)
y_test = np.array(y_test)
print(lg_nrmse(y_test, lasso_preds))

2.009977115814443


In [3]:


from sklearn.linear_model import SGDRegressor
sg = MultiOutputRegressor(SGDRegressor(max_iter = 1000, random_state = 0))
sg.fit(x_train, y_train)
sg_preds = sg.predict(x_test)
y_test = np.array(y_test)
print(lg_nrmse(y_test, sg_preds))

2.014258847055671


In [14]:

# lightgbm 옵션 변경해보기
from lightgbm import LGBMRegressor
import lightgbm
lgb_reg = MultiOutputRegressor(LGBMRegressor(n_estimators = 300, learning_rate = 0.03)).fit(X, y)
lgb_preds = lgb_reg.predict(test)


In [18]:

submit = pd.read_csv('sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = lgb_preds[:,idx-1]
submit.to_csv('D:/자율주행ai/submit0818_1.csv', index = False)

In [7]:
import xgboost
from xgboost import XGBRegressor
xgb_reg = MultiOutputRegressor(XGBRegressor(n_estimators = 300, learning_rate = 0.03)).fit(x_train, y_train)
xgb_preds = xgb_reg.predict(test)


C:\Users\kgt08\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [8]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
rdm = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, min_samples_leaf = 8) ).fit(x_train, y_train) #이거는 한 20분걸림 
preds_rdm = rdm.predict(test)


In [12]:

test = pd.read_csv('test.csv')
test.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_47', 'X_48', 'X_37', 'ID'], axis = 1, inplace = True) 
test = sc.transform(test)

In [86]:
print(X.shape, y.shape)

(39608, 49) (39607, 14)


In [13]:
test

array([[0.42855141, 1.        , 0.617737  , ..., 0.70502544, 0.74847877,
        0.75036626],
       [0.39286215, 0.        , 0.39449541, ..., 0.72259053, 0.75181443,
        0.7676889 ],
       [0.46427571, 0.        , 0.382263  , ..., 0.70723872, 0.71528637,
        0.69520684],
       ...,
       [0.46427571, 0.        , 0.22018349, ..., 0.68137123, 0.76558778,
        0.79377801],
       [0.39286215, 1.        , 0.16207951, ..., 0.70232827, 0.74532527,
        0.82524034],
       [0.53568927, 0.        , 0.20183486, ..., 0.71833873, 0.79444622,
        0.74752395]])

In [83]:
add = X.iloc[-1,:]
X = X.append(add, ignore_index = True)

C:\Users\kyeongtae\AppData\Local\Temp\ipykernel_1568\4064839527.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(add, ignore_index = True)


In [5]:
test = pd.read_csv('test.csv')
test.drop(['X_37', 'ID'], axis = 1, inplace = True) 
test = sc.transform(test);test

array([[0.42855141, 1.        , 0.617737  , ..., 0.70502544, 0.74847877,
        0.75036626],
       [0.39286215, 0.        , 0.39449541, ..., 0.72259053, 0.75181443,
        0.7676889 ],
       [0.46427571, 0.        , 0.382263  , ..., 0.70723872, 0.71528637,
        0.69520684],
       ...,
       [0.46427571, 0.        , 0.22018349, ..., 0.68137123, 0.76558778,
        0.79377801],
       [0.39286215, 1.        , 0.16207951, ..., 0.70232827, 0.74532527,
        0.82524034],
       [0.53568927, 0.        , 0.20183486, ..., 0.71833873, 0.79444622,
        0.74752395]])

In [92]:
# 최적화 코드
# light gbm 사용하기
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import RobustScaler

dataset = pd.read_csv('train.csv')
X = dataset.iloc[:,1:57]
y = dataset.iloc[:,57:]
add = X.iloc[-1,:]
X = X.append(add, ignore_index = True)
addy = y.iloc[-1,:]
y = y.append(addy, ignore_index = True)
X.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_47', 'X_48', 'X_37'], axis = 1, inplace = True) 
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)




# 훈련데이터, 테스트 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)



# 평가방식
from sklearn import metrics
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

test = pd.read_csv('test.csv')
test.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_47', 'X_48', 'X_37', 'ID'], axis = 1, inplace = True) 
test = sc.transform(test)


x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
from lightgbm import LGBMRegressor
import lightgbm
lgb_reg = MultiOutputRegressor(LGBMRegressor(n_estimators = 300, learning_rate = 0.03)).fit(X, y)

import xgboost
from xgboost import XGBRegressor
xgb_reg = MultiOutputRegressor(XGBRegressor(n_estimators = 300, learning_rate = 0.03)).fit(X, y)

# 랜덤 포레스트
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
rdm = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, min_samples_leaf = 8) ).fit(X, y) #이거는 한 20분걸림 

from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators = [('lr', lgb_reg), ('xr', xgb_reg), ('rr', rdm)], voting = 'soft')
voting_clf.fit(x,y)
vot_preds = voting_clf.predict(test)



C:\Users\kyeongtae\AppData\Local\Temp\ipykernel_1568\2552258699.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(add, ignore_index = True)
C:\Users\kyeongtae\AppData\Local\Temp\ipykernel_1568\2552258699.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y = y.append(addy, ignore_index = True)


ValueError: Unknown label type: 'continuous-multioutput'

In [100]:
y_train = y_train.astype('int')
voting_clf.fit(x_train,np.array(y_train).reshape(-1,1))
vot_preds = voting_clf.predict(test)


C:\Users\kyeongtae\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kyeongtae\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: The estimator MultiOutputRegressor should be a classifier.

In [97]:
test

array([[ 0.034643  ,  2.23898549,  1.52273986, ...,  0.31236162,
        -0.7244662 , -1.20891282],
       [-0.34902625, -0.44663085,  0.10555612, ...,  0.8092448 ,
        -0.64953742, -0.78297134],
       [ 0.41868875, -0.44663085,  0.02790222, ...,  0.37497116,
        -1.47006557, -2.56521227],
       ...,
       [ 0.41868875, -0.44663085, -1.00101201, ..., -0.35677077,
        -0.34014718, -0.14147357],
       [-0.34902625,  2.23898549, -1.36986805, ...,  0.23606396,
        -0.79530306,  0.63214484],
       [ 1.18640376, -0.44663085, -1.11749286, ...,  0.68896938,
         0.30809862, -1.27880157]])

In [6]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = [lgb_reg, xgb_reg, rdm], # 선형회귀모형
                                 n_estimators = 5, # 5개의 샘플링
                                 verbose = 1) # 학습 과정 표시
model2 = bagging_model.fit(x_train, y_train) # 학습 진행
predict2 = model2.predict(x_test)

NameError: name 'xgb_reg' is not defined

In [9]:
# 스태킹방법
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [10]:
# soft voting, stacking 이용해서 다시 해보기
from sklearn.ensemble import VotingRegressor

models = [('lgb_reg', lgb_reg), ('xgb_reg', xgb_reg), ('rdm', rdm)]
voting_regressor = VotingRegressor(single_models, voting = 'soft', n_jobs=-1)
voting_regressor.fit(x_train, y_train)
voting_pred = voting_regressor.predict(x_test)


In [ ]:
# 최적화 코드
# light gbm 사용하기
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.multioutput import MultiOutputRegressor

dataset = pd.read_csv('train.csv')
X = dataset.iloc[:,1:57]
y = dataset.iloc[:,57:]

X.drop(['X_37'], axis = 1, inplace = True) 

sc = MinMaxScaler()
X = sc.fit_transform(X)

# 훈련데이터, 테스트 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)



# 평가방식
from sklearn import metrics
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score


from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean
from sklearn.ensemble import VotingRegressor

from lightgbm import LGBMRegressor
import lightgbm
lgb_reg = MultiOutputRegressor(LGBMRegressor(n_estimators = 300, learning_rate = 0.03)).fit(x_train, y_train)

import xgboost
from xgboost import XGBRegressor
xgb_reg = MultiOutputRegressor(XGBRegressor(n_estimators = 300, learning_rate = 0.03)).fit(x_train, y_train)

# 랜덤 포레스트
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
rdm = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, min_samples_leaf = 8)).fit(x_train, y_train) #이거는 한 20분걸림 

models = [('lgb_reg', lgb_reg), ('xgb_reg', xgb_reg), ('rdm', rdm)]
voting_regressor = VotingRegressor(models, n_jobs=-1)
voting_regressor.fit(x_train, y_train)
voting_pred = voting_regressor.predict(x_test)